In [ ]:
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import PIL.Image
import numpy as np


import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline
import imageio

import os
import pickle
import config
import gzip
import json
from tqdm import tqdm_notebook, tqdm
import warnings
import cv2
from keras.models import load_model 

In [1]:
import tensorflow as tf
import os

In [3]:
os.environ['KERAS_BACKEND']='tensorflow' # can choose theano, tensorflow, cntk
os.environ['THEANO_FLAGS']='floatX=float32,device=cpu,optimizer=fast_run,dnn.library_path=/usr/lib'

import keras.backend as K
if os.environ['KERAS_BACKEND'] =='theano':
    channel_axis=1
    K.set_image_data_format('channels_first')
    channel_first = True
else:
    K.set_image_data_format('channels_last')
    channel_axis=-1
    channel_first = False

In [16]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, Dropout, DepthwiseConv2D
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten,PReLU
from keras.layers import Concatenate,Multiply, Add, UpSampling2D,Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal

In [67]:
def __conv_init(a):
    print("conv_init", a)
    k = RandomNormal(0, 0.02)(a) # for convolution kernel
    k.conv_weight = True    
    return k

In [68]:
# Basic discriminator
def conv2d(f, *a, **k):
    return Conv2D(f, kernel_initializer = conv_init, *a, **k)
def batchnorm():
    return BatchNormalization(momentum=0.9, axis=channel_axis, epsilon=1.01e-5,
                                   gamma_initializer = gamma_init)
def BASIC_D(nc_in, ndf, max_layers=3, use_sigmoid=True):
    """DCGAN_D(nc, ndf, max_layers=3)
       nc: channels
       ndf: filters of the first layer
       max_layers: max hidden layers
    """    
    if channel_first:
        input_a =  Input(shape=(nc_in, None, None))
    else:
        input_a = Input(shape=(None, None, nc_in))
    _ = input_a
    _ = conv2d(ndf, kernel_size=4, strides=2, padding="same", name = 'First') (_)
    _ = LeakyReLU(alpha=0.2)(_)
    
    for layer in range(1, max_layers):        
        out_feat = ndf * min(2**layer, 8)
        _ = conv2d(out_feat, kernel_size=4, strides=2, padding="same", 
                   use_bias=False, name = 'pyramid.{0}'.format(layer)             
                        ) (_)
        _ = batchnorm()(_, training=1)        
        _ = LeakyReLU(alpha=0.2)(_)
    
    out_feat = ndf*min(2**max_layers, 8)
    _ = ZeroPadding2D(1)(_)
    _ = conv2d(out_feat, kernel_size=4,  use_bias=False, name = 'pyramid_last') (_)
    _ = batchnorm()(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    
    # final layer
    _ = ZeroPadding2D(1)(_)
    _ = conv2d(1, kernel_size=4, name = 'final'.format(out_feat, 1), 
               activation = "sigmoid" if use_sigmoid else None) (_)    
    return Model(inputs=[input_a], outputs=_)



In [69]:
conv_init = RandomNormal(0, 0.02)
gamma_init = RandomNormal(1., 0.02) 


In [6]:
from keras.layers import Layer, InputSpec
from keras import initializers, regularizers, constraints
from keras import backend as K


class InstanceNormalization(Layer):
    """Instance normalization layer.
    Normalize the activations of the previous layer at each step,
    i.e. applies a transformation that maintains the mean activation
    close to 0 and the activation standard deviation close to 1.
    # Arguments
        axis: Integer, the axis that should be normalized
            (typically the features axis).
            For instance, after a `Conv2D` layer with
            `data_format="channels_first"`,
            set `axis=1` in `InstanceNormalization`.
            Setting `axis=None` will normalize all values in each
            instance of the batch.
            Axis 0 is the batch dimension. `axis` cannot be set to 0 to avoid errors.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
            When the next layer is linear (also e.g. `nn.relu`),
            this can be disabled since the scaling
            will be done by the next layer.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a Sequential model.
    # Output shape
        Same shape as input.
    # References
        - [Layer Normalization](https://arxiv.org/abs/1607.06450)
        - [Instance Normalization: The Missing Ingredient for Fast Stylization](
        https://arxiv.org/abs/1607.08022)
    """
    def __init__(self,
                 axis=-1,
                 epsilon=1e-8,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):
        ndim = len(input_shape)
        if self.axis == 0:
            raise ValueError('Axis cannot be zero')

        if (self.axis is not None) and (ndim == 2):
            raise ValueError('Cannot specify axis for rank 1 tensor')

        self.input_spec = InputSpec(ndim=ndim)

        if self.axis is None:
            shape = (1,)
        else:
            shape = (input_shape[self.axis],)

        if self.scale:
            self.gamma = self.add_weight(shape=shape,
                                         name='gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.gamma_constraint)
        else:
            self.gamma = None
        if self.center:
            self.beta = self.add_weight(shape=shape,
                                        name='beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constraint)
        else:
            self.beta = None
        self.built = True

    def call(self, inputs, training=None):
        input_shape = K.int_shape(inputs)
        reduction_axes = list(range(0, len(input_shape)))

        if self.axis is not None:
            del reduction_axes[self.axis]

        del reduction_axes[0]

        mean = K.mean(inputs, reduction_axes, keepdims=True)
        stddev = K.std(inputs, reduction_axes, keepdims=True) + self.epsilon
        normed = (inputs - mean) / stddev

        broadcast_shape = [1] * len(input_shape)
        if self.axis is not None:
            broadcast_shape[self.axis] = input_shape[self.axis]

        if self.scale:
            broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
            normed = normed * broadcast_gamma
        if self.center:
            broadcast_beta = K.reshape(self.beta, broadcast_shape)
            normed = normed + broadcast_beta
        return normed

    def get_config(self):
        config = {
            'axis': self.axis,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint)
        }
        base_config = super(InstanceNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))




In [13]:
def normalize():
#   return BatchNormalization(axis=get_filter_dim())
    return InstanceNormalization()

def scaleup(input, ngf, kss, strides, padding):
#   x = Conv2DTranspose(ngf, kss, strides=strides, padding=padding)(input)

    # upsample + conv
    x = UpSampling2D(strides)(input)
    x = Conv2D(ngf, kss, padding=padding)(x)
    return x


def res_block(input, filters, kernel_size=(3,3), strides=(1,1)):
    # conv_block:add(nn.SpatialReflectionPadding(1, 1, 1, 1))
    # conv_block:add(nn.SpatialConvolution(dim, dim, 3, 3, 1, 1, p, p))
    # conv_block:add(normalization(dim))
    # conv_block:add(nn.ReLU(true))
    
    x = input
    x = Lambda(lambda x: tf.pad(x, [[0,0], [1,1], [1,1], [0,0]], 'REFLECT'))(x)
    x = Conv2D(filters=filters, 
              kernel_size=kernel_size,
              strides=strides)(x)
    x = normalize()(x)
    x = Activation('relu')(x)

    x = Lambda(lambda x: tf.pad(x, [[0,0], [1,1], [1,1], [0,0]], 'REFLECT'))(x)
    x = Conv2D(filters=filters, 
              kernel_size=kernel_size,
              strides=strides)(x)
    x = normalize()(x)

#   merged = Concatenate(axis=get_filter_dim())([input, x])
    merged = Add()([input, x])
    return merged

def resnet_6blocks(input_shape, output_nc, ngf, **kwargs):
    ks = 3
    f = 7
    p = int((f-1)/2)

    input = Input(input_shape)
    # local e1 = data - nn.SpatialReflectionPadding(p, p, p, p) - nn.SpatialConvolution(3, ngf, f, f, 1, 1) - normalization(ngf) - nn.ReLU(true)
    x = input
    x = Lambda(lambda x: tf.pad(x, [[0,0], [3,3], [3,3], [0,0]], 'REFLECT'))(x)
    x = Conv2D(ngf, (f,f))(x)
    x = normalize()(x)
    x = Activation('relu')(x)

    # local e2 = e1 - nn.SpatialConvolution(ngf, ngf*2, ks, ks, 2, 2, 1, 1) - normalization(ngf*2) - nn.ReLU(true)
    x = Conv2D(ngf*2, (ks,ks), strides=(2,2), padding='same')(x)
    x = normalize()(x)
    x = Activation('relu')(x)

    # local e3 = e2 - nn.SpatialConvolution(ngf*2, ngf*4, ks, ks, 2, 2, 1, 1) - normalization(ngf*4) - nn.ReLU(true)
    x = Conv2D(ngf*4, (ks,ks), strides=(2,2), padding='same')(x)
    x = normalize()(x)
    x = Activation('relu')(x)

    # local d1 = e3 - build_res_block(ngf*4, padding_type) - build_res_block(ngf*4, padding_type) - build_res_block(ngf*4, padding_type) 
    #  - build_res_block(ngf*4, padding_type) - build_res_block(ngf*4, padding_type) - build_res_block(ngf*4, padding_type)
    x = res_block(x, ngf*4)
    x = res_block(x, ngf*4)
    x = res_block(x, ngf*4)
    x = res_block(x, ngf*4)
    x = res_block(x, ngf*4)
    x = res_block(x, ngf*4)

    # local d2 = d1 - nn.SpatialFullConvolution(ngf*4, ngf*2, ks, ks, 2, 2, 1, 1,1,1) - normalization(ngf*2) - nn.ReLU(true)
    # x = Conv2DTranspose(ngf*2, (ks,ks), strides=(2,2), padding='same')(x)
    x = scaleup(x, ngf*2, (ks, ks), strides=(2,2), padding='same')
    x = normalize()(x)
    x = Activation('relu')(x)
    
    # local d3 = d2 - nn.SpatialFullConvolution(ngf*2, ngf, ks, ks, 2, 2, 1, 1,1,1) - normalization(ngf) - nn.ReLU(true)
    # x = Conv2DTranspose(ngf, (ks,ks), strides=(2,2), padding='same')(x)
    x = scaleup(x, ngf, (ks, ks), strides=(2,2), padding='same')
    x = normalize()(x)
    x = Activation('relu')(x)

    # local d4 = d3 - nn.SpatialReflectionPadding(p, p, p, p) - nn.SpatialConvolution(ngf, output_nc, f, f, 1, 1) - nn.Tanh()
    x = Lambda(lambda x: tf.pad(x, [[0,0], [3,3], [3,3], [0,0]], 'REFLECT'))(x)
    x = Conv2D(output_nc, (f,f))(x)
    x = Activation('tanh')(x)
    
    model = Model(input, [x], name=kwargs.get('name',None))
    print('Model resnet 6blocks:')
    model.summary()
    return model

In [32]:

def res_block(input, filters, kernel_size=(3,3), strides=(1,1)):
    
    x = input
    x = Lambda(lambda x: tf.pad(x, [[0,0], [1,1], [1,1], [0,0]], 'REFLECT'))(x)
    x = Conv2D(filters=filters, 
              kernel_size=kernel_size,
              strides=strides)(x)
    x = normalize()(x)
    x = Activation('relu')(x)

    x = Lambda(lambda x: tf.pad(x, [[0,0], [1,1], [1,1], [0,0]], 'REFLECT'))(x)
    x = Conv2D(filters=filters, 
              kernel_size=kernel_size,
              strides=strides)(x)
    x = normalize()(x)

    merged = Add()([input, x])
    return merged

In [70]:
def generator_sxt(input_shape=(512,512,4), output_nc=4, ngf=64):
    
    ks = 3
    f = 7
    p = int((f-1)/2)
    
    list_info_s =[]
    list_info_t =[]

    input_s = Input(input_shape)
    input_t = Input(input_shape)
    
    s_layer = input_s
    
    s_layer = Conv2D(ngf, (f,f))(s_layer)
    s_layer = normalize()(s_layer)
    s_layer = Activation('relu')(s_layer)
    
    s_layer = Conv2D(ngf*2, (3,3), strides=(2,2), padding='same')(s_layer)
    s_layer = normalize()(s_layer)
    s_layer = Activation('relu')(s_layer)
    
    list_info_s.append(s_layer)

    s_layer = Conv2D(ngf*4, (3,3), strides=(2,2), padding='same')(s_layer)
    s_layer = normalize()(s_layer)
    s_layer = Activation('relu')(s_layer)
    
    list_info_s.append(s_layer)
    
    
    t_layer = input_t
    
    t_layer = Conv2D(ngf, (f,f))(t_layer)
    t_layer = normalize()(t_layer)
    t_layer = Activation('relu')(t_layer)
    
    t_layer = Conv2D(ngf*2, (3,3), strides=(2,2), padding='same')(t_layer)
    t_layer = normalize()(t_layer)
    t_layer = Activation('relu')(t_layer)
    
    list_info_t.append(t_layer)

    t_layer = Conv2D(ngf*4, (3,3), strides=(2,2), padding='same')(t_layer)
    t_layer = normalize()(t_layer)
    t_layer = Activation('relu')(t_layer)
    
    list_info_t.append(t_layer)
    
    s_layer = Conv2D(ngf*2, (3,3), strides=(1,1), padding='same')(s_layer)
    t_layer = Conv2D(ngf*2, (3,3), strides=(1,1), padding='same')(t_layer)
    r_s_layer = r_t_layer = Multiply()([s_layer,t_layer])
    
    s_layer = Concatenate()([s_layer,r_s_layer])
    t_layer = Concatenate()([t_layer,r_t_layer])
    
    s_layer = res_block(s_layer, ngf*4)
    t_layer = res_block(t_layer, ngf*4)
    
    s_layer = t_layer = Multiply()([s_layer,t_layer])
    
    s_layer = res_block(s_layer, ngf*4)
    t_layer = res_block(t_layer, ngf*4)
    
    s_layer = t_layer = Multiply()([s_layer,t_layer])
    
    s_layer = res_block(s_layer, ngf*4)
    t_layer = res_block(t_layer, ngf*4)
    
    
    s_layer = Concatenate()([s_layer,list_info_s[1]])
    t_layer = Concatenate()([t_layer,list_info_t[1]])
    
    s_layer = Conv2DTranspose(ngf*2, (ks,ks), strides=(2,2), padding='same')(s_layer)
    s_layer = normalize()(s_layer)
    s_layer = Activation('relu')(s_layer)
    
    t_layer = Conv2DTranspose(ngf*2, (ks,ks), strides=(2,2), padding='same')(t_layer)
    t_layer = normalize()(t_layer)
    t_layer = Activation('relu')(t_layer)
    
    s_layer_0 = Lambda(lambda x: tf.pad(x, [[0,0], [1,0], [1,0], [0,0]], 'REFLECT'))(list_info_s[0])
    t_layer_0 = Lambda(lambda x: tf.pad(x, [[0,0], [1,0], [1,0], [0,0]], 'REFLECT'))(list_info_t[0])
    
    s_layer = Concatenate()([s_layer,s_layer_0])
    t_layer = Concatenate()([t_layer,t_layer_0])
    
    c_layer = Concatenate()([t_layer,s_layer])
    
    c_layer = Lambda(lambda x: tf.pad(x, [[0,0], [2,0], [2,0], [0,0]], 'REFLECT'))(c_layer)
    c_layer = Conv2DTranspose(ngf, (ks,ks), strides=(2,2), padding='same')(c_layer)
    c_layer = normalize()(c_layer)
    c_layer = Activation('relu')(c_layer)
    
    
    c_layer = Lambda(lambda x: tf.pad(x, [[0,0], [3,3], [3,3], [0,0]], 'REFLECT'))(c_layer)
    c_layer = Conv2D(output_nc, (f,f))(c_layer)
    c_layer = Activation('tanh')(c_layer)

    
    model = Model([input_s,input_t], c_layer)
    model.summary()

    return model
    
    
    

In [71]:
m=generator_sxt()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_51 (InputLayer)           (None, 512, 512, 4)  0                                            
__________________________________________________________________________________________________
input_52 (InputLayer)           (None, 512, 512, 4)  0                                            
__________________________________________________________________________________________________
conv2d_410 (Conv2D)             (None, 506, 506, 64) 12608       input_51[0][0]                   
__________________________________________________________________________________________________
conv2d_413 (Conv2D)             (None, 506, 506, 64) 12608       input_52[0][0]                   
__________________________________________________________________________________________________
instance_n

In [29]:
netDA= gram_net_discriminator()
netDB= gram_net_discriminator()
netDA.summary()

256

In [ ]:
ngf=64
netGB=resnet_6blocks((512,512,3),3,ngf)
netGA=resnet_6blocks((512,512,3),3,ngf)
netGA.summary()

In [ ]:
from keras.optimizers import RMSprop, SGD, Adam
if use_lsgan:
    loss_fn = lambda output, target : K.mean(K.abs(K.square(output-target)))
else:
    loss_fn = lambda output, target : -K.mean(K.log(output+1e-12)*target+K.log(1-output+1e-12)*(1-target))


def cycle_variables(netG1, netG2):
    real_input_s = netG1.inputs[0]
    fake_output_s = netG1.outputs[0]
    
    real_input_t = netG1.inputs[1]
    
    rec_input = netG2([fake_output_s,real_input_s])
    
    fn_generate = K.function([real_input_s,real_input_t], [fake_output_s, rec_input])
    return real_input_s,real_input_t,fake_output_s, rec_input, fn_generate

real_A_s, real_A_t, fake_B, rec_A, cycleA_generate  = cycle_variables(netGB, netGA)
real_B_s, real_B_t, fake_A, rec_B, cycleB_generate = cycle_variables(netGA, netGB)

In [ ]:
def perceptual_loss00(y_true, y_pred): 
    n_v = tf.constant([[[[103.939, 116.779, 123.68]]]])
    y_true = ((y_true[...,::2,::2,::-1]+1)*127.5 - n_v)
    y_pred = ((y_pred[...,::2,::2,::-1]+1)*127.5 - n_v)

    
    true_loss = loss_model(y_true)
    pred_loss = loss_model(y_pred)
    loss = 0
    loss +=1*K.mean(K.abs(y_pred - y_true))
    loss += 0.001*K.mean(K.square(true_loss[0] - pred_loss[0]))
    loss += 0.0005*K.mean(K.square(true_loss[1] - pred_loss[1]))
    return loss

from keras.applications.vgg16 import VGG16, preprocess_input

vgg = VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3)) 
loss_model = Model(inputs=vgg.input, outputs=[vgg.get_layer('block1_conv2').output,
                                              vgg.get_layer('block2_conv2').output]) 
loss_model.trainable = False

In [ ]:
def D_loss(netD, real_s,real_t, fake, rec):
    output_real = netD([real_s])
    output_fake = netD([fake])
    loss_D_real = loss_fn(output_real, K.ones_like(output_real))
    loss_D_fake = loss_fn(output_fake, K.zeros_like(output_fake))
    loss_G = loss_fn(output_fake, K.ones_like(output_fake))
    loss_D = loss_D_real+loss_D_fake
    loss_cyc = K.mean(K.abs(rec-real))+1.*perceptual_loss00(real,rec)

    return loss_D, loss_G, loss_cyc

loss_DA, loss_GA, loss_cycA = D_loss(netDA, real_A_s, real_A_t, fake_B, rec_A)
loss_DB, loss_GB, loss_cycB = D_loss(netDB, real_B_s, real_B_t, fake_A, rec_B)
loss_cyc = loss_cycA+loss_cycB


In [ ]:
loss_G = loss_GA+loss_GB+λ*loss_cyc
loss_D = loss_DA+loss_DB

weightsD = netDA.trainable_weights + netDB.trainable_weights
weightsG = netGA.trainable_weights + netGB.trainable_weights

training_updates = Adam(lr=lrD*0.1, beta_1=0.5).get_updates(weightsD,[],loss_D)
netD_train = K.function([real_A, real_B],[loss_DA/2, loss_DB/2], training_updates)
training_updates = Adam(lr=lrG*0.1, beta_1=0.5).get_updates(weightsG,[], loss_G)
netG_train = K.function([real_A, real_B], [loss_GA, loss_GB, loss_cyc], training_updates)


In [ ]:
def minibatch(s_data,t_data, batchsize):
    length = len(data)
    epoch = i = 0
    tmpsize = None    
    while True:
        size = tmpsize if tmpsize else batchsize
        if i+size > length:
            shuffle(data)
            i = 0
            epoch+=1   
        idx = np.random.randint(0,data.shape[0], size)
        rtn0 = s_data[idx] 
        rtn1 = t_data[idx] 
        i+=size
        tmpsize = yield epoch, np.float32([rtn0,rtn1])       

def minibatchAB(dataA, dataB, batchsize):
    batchA=minibatch(dataA,dataB, batchsize)
    batchB=minibatch(dataB,dataA, batchsize)
    tmpsize = None    
    while True:        
        ep1, A = batchA.send(tmpsize)
        ep2, B = batchB.send(tmpsize)
        tmpsize = yield max(ep1, ep2), A, B

In [ ]:

plt.rcParams['figure.figsize'] = (30,30)
def showX(X, rows=1):
    assert X.shape[0]%rows == 0
    int_X = ( (X+1)/2*255).clip(0,255).astype('uint8')
    if channel_first:
        int_X = np.moveaxis(int_X.reshape(-1,3,imageSize,imageSize), 1, 3)
    else:
        int_X = int_X.reshape(-1,imageSize,imageSize, 3)
    int_X = int_X.reshape(rows, -1, imageSize, imageSize,3).swapaxes(1,2).reshape(rows*imageSize,-1, 3)
    plt.imshow(int_X.astype('uint8'))
    plt.show()
    

In [ ]:
train_batch = minibatchAB(train_A, train_B, 6)

_, A, B = next(train_batch)
showX(A)
showX(B)

del train_batch, A, B

In [ ]:
def showG(A,B):
    assert A.shape==B.shape
    def G(fn_generate, X,Y):

        r = np.array([fn_generate([X[i:i+1],Y[i:i+1]]) for i in range(X.shape[0])])
        return r.swapaxes(0,1)[:,:,0]        
    rA = G(cycleA_generate, A,B)
    rB = G(cycleB_generate, B,A)
    arr = np.concatenate([A,B,rA[0],rB[0],rA[1],rB[1]])
    #new = np.concatenate([rA[0][0],rB[0][0]],axis=1)
    #plt.imshow(( (new+1)/2*255).clip(0,255).astype('uint8'))
    #plt.show()
    showX(arr, 3)

In [ ]:
x_data = np.memmap('x_data.npy', dtype='float32', mode='r', shape=(len(find_x_data),512,512,4))[:,:,:,:3]
y_data = np.memmap('y_data.npy', dtype='float32', mode='r', shape=(len(find_x_data),512,512,3))

In [ ]:
import time
from IPython.display import clear_output
t0 = time.time()

std_loss_A = []
mean_loss_A =[]
std_loss_B = []
mean_loss_B =[]
errCyc_sum = errGA_sum = errGB_sum = errDA_sum = errDB_sum = m_loss_A_sum = a_loss_A_sum=m_loss_B_sum = a_loss_B_sum = 0


display_iters = 25
#val_batch = minibatch(valAB, 6, direction)

niter = 1500
gen_iterations = 0
epoch = 1200
batchSize = 4
train_batch = minibatchAB(x_data, y_data, batchSize)


In [ ]:
def calculte_fid(real_data,fake_data,fid_batch=512, id0 = 0):
  
        
    fid_image_real = []
    fid_image_fake = []
    select_real=np.random.randint(0,real_data.shape[0], fid_batch)
    select_fake=np.random.randint(0,fake_data.shape[0], fid_batch)
      
    for range_latent_real in real_data[select_real]:
        range_image_real = correct_warp_with_landmarks(cv2.resize(my_Gs.components.synthesis.run(range_latent_real[None,...].reshape(-1,18,512), randomize_noise=False, **synthesis_kwargs)[0],(256,256)))[None,...]/255*2-1 

        if id0==0:
            rA =  netGA.predict(range_image_real)[0]
        else:
            rA = netGB.predict(range_image_real)[0]

      
        fid_image_fake.append(((rA+1)/2*255).clip(0,255).astype('uint8'))
        
    for range_latent_fake in fake_data[select_fake]:
        fid_image_real.append(correct_warp_with_landmarks(cv2.resize(my_Gs.components.synthesis.run(range_latent_fake[None,...].reshape(-1,18,512), randomize_noise=False, **synthesis_kwargs)[0],(256,256))))
        
    fid_image_fake = np.array(fid_image_fake).astype(np.float32)
    fid_image_real = np.array(fid_image_real).astype(np.float32)

    fake_images = scale_images(fid_image_fake,(512,512,3))
    new_real = scale_images(fid_image_real,(512,512,3))
    images1 = preprocess_input(new_real)
    images2 = preprocess_input(fake_images)
      

    fid = calculate_fid_0(model_fid, images1, images2)
    print(fid)
    return fid

def calculate_fid_0(model, images1, images2):
    # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = numpy.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
     # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid
    
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, 0)
        # store
        images_list.append(new_image)
    return asarray(images_list)

In [ ]:
percept_loss = []
fid_A_loss = []
fid_B_loss = []

In [ ]:
errCyc_sum = errGA_sum = errGB_sum = errDA_sum = errDB_sum=m_loss_A_sum = a_loss_A_sum=m_loss_B_sum = a_loss_B_sum = 0
        
display_iters = 200
#val_batch = minibatch(valAB, 6, direction)

niter = 1500
gen_iterations = 0
epoch = 1200
batchSize = 4
train_batch = minibatchAB(train_A, train_B, batchSize)


while epoch < niter: 
    epoch, A, B = next(train_batch)        
    errDA, errDB  = netD_train([A, B])
    errDA_sum +=errDA
    errDB_sum +=errDB

    # epoch, trainA, trainB = next(train_batch)
    errGA, errGB, errCyc = netG_train([A, B])
    
    errGA_sum += errGA
    errGB_sum += errGB
    errCyc_sum += errCyc
    
    gen_iterations+=1

    if gen_iterations%display_iters==0:
        clear_output()
        print('[%d/%d][%d] Loss_D: %f %f Loss_G: %f %f loss_cyc %f '
        % (epoch, niter, gen_iterations, errDA_sum/display_iters, errDB_sum/display_iters,
           errGA_sum/display_iters, errGB_sum/display_iters, 
           errCyc_sum/display_iters), time.time()-t0)
        _, A, B = train_batch.send(3)
        percept_loss.append(errCyc_sum/display_iters)
        showG(A,B)        
        errCyc_sum = errGA_sum = errGB_sum = errDA_sum = errDB_sum=m_loss_A_sum = a_loss_A_sum=m_loss_B_sum = a_loss_B_sum = 0
        plt.plot(percept_loss[1:])
        plt.show()

    if gen_iterations%100==0:

        test_image_s0 = cv2.resize(imageio.imread('test/before_AH10x_0221_011_IMG_20200410_150014.jpg'),(512,512))[None,...]/255*2-1
        test_image_t0 = cv2.resize(imageio.imread('test/after_AH10x_0221_011_IMG_20200410_150014.jpg'),(512,512))[None,...]/255*2-1
        test_image_s1 = cv2.resize(imageio.imread('test/before_MJ10x_0024_IMG_20191226_135929_23_0_003_013_007_BE6I8878_8.2x.jpg'),(512,512))[None,...]/255*2-1
        test_image_t1 = cv2.resize(imageio.imread('test/after_MJ10x_0024_IMG_20191226_135929.jpg'),(512,512))[None,...]/255*2-1

        rA0 = netGB.predict([test_image_s0,test_image_t0])
        rB0 = netGA.predict([test_image_t0,test_image_s0])
        rA1 = netGB.predict([test_image_s1,test_image_t1])
        rB1 = netGA.predict([test_image_t1,test_image_s1])
        
        test0 = np.concatenate([((test_image_s0[0]+1)/2*255)clip(0,255).astype('uint8'),((test_image_t0[0]+1)/2*255)clip(0,255).astype('uint8')],axis=0)
        test0 = np.concatenate([test0,cv2.resize(((rA0[0]+1)/2*255)clip(0,255).astype('uint8'),(1024,1024))],axis=1)
        
        test1 = np.concatenate([((test_image_t0[0]+1)/2*255)clip(0,255).astype('uint8'),((test_image_s0[0]+1)/2*255)clip(0,255).astype('uint8')],axis=0)
        test1 = np.concatenate([test1,cv2.resize(((rB0[0]+1)/2*255)clip(0,255).astype('uint8'),(1024,1024))],axis=1)
        
        
        test2 = np.concatenate([((test_image_s1[0]+1)/2*255)clip(0,255).astype('uint8'),((test_image_t1[0]+1)/2*255)clip(0,255).astype('uint8')],axis=0)
        test2 = np.concatenate([test2,cv2.resize(((rA1[0]+1)/2*255)clip(0,255).astype('uint8'),(1024,1024))],axis=1)
        
        test3 = np.concatenate([((test_image_t1[0]+1)/2*255)clip(0,255).astype('uint8'),((test_image_s1[0]+1)/2*255)clip(0,255).astype('uint8')],axis=0)
        test3 = np.concatenate([test3,cv2.resize(((rB1[0]+1)/2*255)clip(0,255).astype('uint8'),(1024,1024))],axis=1)
        
        plt.imshow(test0)
        plt.show()
        plt.imshow(test1)
        plt.show()
        plt.imshow(test2)
        plt.show()
        plt.imshow(test3)
        plt.show()



